In [ ]:
!pip install roboflow

In [1]:
import torch
from ultralytics import YOLO

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="<API_KEY>")
project = rf.workspace("<WORKSPACE_NAME>").project("<PROJECT_NAME>")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
weights = [
    "yolov8m-face.pt",
    "yolov8n-face.pt",
    "yolov8l-face.pt",
    "yolov5m-face.pt",
    "yolov5n-face.pt",
    "yolov5l-face.pt",
    "yolov8l-face.engine",
]

for weight in weights:
    detection_model = YOLO(weight)
    detection_model = detection_model.to("cuda")

    metrics = detection_model.val(data="Faces-Detection-2/data.yaml")
    print(metrics.box.map50)
    print(metrics.box.map75)
    print(metrics.box.map)

In [12]:
metrics.box.map50, metrics.box.map75, metrics.box.map

(0.9856793261798135, 0.8364813297963953, 0.6537393275038478)

In [3]:
import pandas as pd

index = ["yolov8-m", "yolov8-n", "yolov8-l", "yolov5-m", "yolov5-n", "yolov5-l", "yolov8-l-tensorrt"]
final_metrics = pd.DataFrame({
    "map@50": [0.9821, 0.9826, 0.9857, 0.8213, 0.7966, 0.8301, 0.9826],
    "map@75": [0.8765, 0.8365, 0.8786, 0.7131, 0.6834, 0.7262, 0.8759],
    "map@50:95": [0.6850, 0.6537, 0.7048, 0.6306, 0.6019, 0.6893, 0.7023],
    "time, ms": [20.2, 3.1, 28.1, 9.9, 7.5, 15.9, 6.3]
}, index=index)

final_metrics

,map@50,map@75,map@50:95,"time, ms"
yolov8-m,0.9821,0.8765,0.6850,20.2
yolov8-n,0.9826,0.8365,0.6537,3.1
yolov8-l,0.9857,0.8786,0.7048,28.1
yolov5-m,0.8213,0.7131,0.6306,9.9
yolov5-n,0.7966,0.6834,0.6019,7.5
yolov5-l,0.8301,0.7262,0.6893,15.9
yolov8-l-tensorrt,0.9826,0.8759,0.7023,6.3


### ONNX -> TensorRT + Quantization

In [2]:
model = YOLO("yolov8l-face.pt")
model = model.to("cuda")
model.export(format="engine", batch=16, imgsz=640, dynamic=True, verbose=False, half=True)

Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs

PyTorch: starting from 'yolov8l-face.pt' with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 5, 8400) (83.6 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success  22.5s, saved as 'yolov8l-face.onnx' (166.5 MB)

TensorRT: starting export with TensorRT 10.0.1...
TensorRT: input "images" with shape(-1, 3, -1, -1) DataType.FLOAT
TensorRT: output "output0" with shape(-1, 5, -1) DataType.FLOAT
TensorRT: building FP16 engine as yolov8l-face.engine
TensorRT: export success  964.0s, saved as 'yolov8l-face.engine' (85.7 MB)

Export complete (968.4s)
Results saved to D:\diploma\notebooks
Predict:         yolo predict task=detect model=yolov8l-face.engine imgsz=640 half 
Validate:        yolo val task=detect model=yolov8l-face.engine imgsz=640 data=cfg/datasets/WiderFace.yaml half 
Visualize:       https://netron.app


'yolov8l-face.engine'